In [11]:
myName = "Jing Li"
name_split = myName.split(',')
print(name_split)

['Jing Li']


In [9]:
from datetime import datetime
currTime = datetime.now()
print(currTime)

2019-08-13 23:20:05.739238


In [7]:
currTime(1)

TypeError: 'datetime.datetime' object is not callable

In [12]:
myStr = ['alpha','beta','gamma']
myStr[:-1]

['alpha', 'beta']

In [24]:
import bibtexparser

pubtype_dict = {
    'Uncategorized': '"0"',
    'misc': '"0"',
    'inproceedings': '"1"',
    'conference': '"1"',
    'article': '"2"',
    'submitted': '"3"',
    'preprint': '"3"',
    'techreport': '"4"',
    'book': '"5"',
    'incollection': '"6"',
    'phdthesis': '"7"',
    'mastersthesis': '"7"',
    'patent': '"8"',
}

with open('publications.bib') as bibtex_file:
    bibtex_str = bibtex_file.read()

bib_database = bibtexparser.loads(bibtex_str)
numRef = 0
for entry in bib_database.entries:
    numRef += 1
    print(str(numRef) + ": " + entry['ID'])
    if 'date' in entry:
        if len(entry['date']) == 10:
            print('date: ' + entry['date'])
        else: 
            print('date: ' + entry['date'] + '-01')
#    if 'ENTRYTYPE' in entry:
#        print('publication_types: ['+pubtype_dict[entry['ENTRYTYPE']]+']\n')
#    if 'abstract' in entry:
#        print('abstract: "'+entry['abstract'] +'"\n')


        

1: ratner2019sysml
date: 2019-03-29
2: shukla2018frontiers
date: 2018-02-14
3: zha2018JOLPE
date: 2018-02-15
4: khoram2018CAL
date: 2018-01-03
5: zha2017CALCMA
date: 2017-07-03
6: zha2017CALIMEC
date: 2017-02-22
7: li2014jssc
8: cil2013thinfilm
9: zhang2012TED
10: cywar2012nano
date: 2012-05-10
11: li2011sciencechina
12: li2010tvlsi
13: chen2010tvlsi
14: li2009tcad
15: li2008jetc
16: li2007ted
17: luo2019vlsit
date: 2019-06-09
18: zha2019vlsic
date: 2019-06-09
19: zhang2019fccm
date: 2019-04-28
20: zhang2019fpga
date: 2019-02-24
21: zhang2018fccm
date: 2018-04-29
22: zhang2018cvpr
date: 2018-06-01
23: khoram2018iclr
date: 2018-04-01
24: li2018CSTIC
date: 2018-03-12
25: zha2018asplos
date: 2018-03-19
26: zhang2018fpga
date: 2018-02-25
27: zha2018fpga
date: 2018-02-25
28: khoram2018fpga
date: 2018-02-25
29: zha2017iccad
date: 2017-11-13
30: khoram2017fccm
date: 2017-04-30
31: khoram2017ISPD
date: 2017-03-19
32: zha2017CSTIC
date: 2017-03-12
33: zha2017dacwip
date: 2017-06-01
34: zhang201

In [108]:
import re
   
def removeEmptyString(inStr):
    while("" in inStr):
        inStr.remove("") 
    
str1 = 'meza2012:report:nvm'
if ':' in str1:
    str2 = str1.replace(':', "-")
print(str2)

string_to_split = 'li2009phd'
a = re.split('[^a-zA-Z]', string_to_split)
b = re.split('[a-zA-Z]', string_to_split)

#removeEmptyString(a)
#removeEmptyString(b)

#c = re.split('[^A-Z]', a[1])
#c = re.split('[A-Z][a-zA-Z]*', a[1])
#removeEmptyString(c)
#d = re.findall('[A-Z][a-zA-Z]*', a[1])
print(a)
print(b)
#print(c)
#print(d)

#outputStr = '-'.join([a[0]] + b + c + d)

#print(outputStr.lower())


meza2012-report-nvm
['li', '', '', '', 'phd']
['', '', '2009', '', '', '']


In [11]:
testStr = 'zhang2017fpgaCNN'
def splitnonalpha(s):
   pos = 1
   while pos < len(s) and s[pos].isalpha():
      pos+=1
   return (s[:pos], s[pos:])
splitnonalpha(testStr)

('zhang', '2017fpgaCNN')

In [17]:
import datetime

#datetime.datetime.strptime('2018-09-12', '%Y-%m-%d')

help(datetime.datetime)

Help on class datetime in module datetime:

class datetime(date)
 |  datetime(year, month, day[, hour[, minute[, second[, microsecond[,tzinfo]]]]])
 |  
 |  The year, month and day arguments are required. tzinfo may be None, or an
 |  instance of a tzinfo subclass. The remaining arguments may be ints.
 |  
 |  Method resolution order:
 |      datetime
 |      date
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __new__(*args, **kwa

In [26]:
'1'.zfill(2)

'01'